# User Notebook

In [1]:
import pandas as pd
import numpy as np
from data_prep_functions import factorial

6


In [2]:
factorial(21)

51090942171709440000

In [9]:
from ipynb.fs.defs.data_config_funcs import add_seven

ImportError: cannot import name 'add_seven' from 'ipynb.fs.defs.data_config_funcs' (unknown location)